In [3]:
class Bound:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}, {self.y})"

    def __contains__(self, item):
        i, j = item
        return 0 <= i < self.x and 0 <= j < self.y

    def __iter__(self):
        for i in range(self.x):
            for j in range(self.y):
                yield (i, j)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def limit(self, item):
        i, j = item
        return max(0, min(i, self.x - 1)), max(0, min(j, self.y - 1))

In [15]:
with open('input15.txt', 'r') as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]

# The entire cave is actually five times larger in both dimensions than you thought; the area you originally scanned is just one tile in a 5x5 tile area that forms the full map. Your original map tile repeats to the right and downward; each time the tile repeats to the right or downward, all of its risk levels are 1 higher than the tile immediately up or left of it. However, risk levels above 9 wrap back around to 1. So, if your original map had some position with a risk level of 8, then that same position on each of the 25 total tiles would be as follows:

# 8 9 1 2 3
# 9 1 2 3 4
# 1 2 3 4 5
# 2 3 4 5 6
# 3 4 5 6 7

# Each single digit above corresponds to the example position with a value of 8 on the top-left tile. Because the full map is actually five times larger in both dimensions, that position appears a total of 25 times, once in each duplicated tile, with the values shown above.

import numpy as np

grid = np.array(grid)
actual_grid = np.empty((5*len(grid), 5*len(grid[0])), dtype=int)
for i in range(5):
    for j in range(5):
        si, sj = i*len(grid), j*len(grid[0])
        ei, ej = si + len(grid), sj + len(grid[0])
        actual_grid[si:ei, sj:ej] = (grid + i + j - 1) % 9 + 1
        
grid = actual_grid.tolist()

In [16]:
# find cost of minimum path from (0,0) to bottom right

from heapq import heappush, heappop

def get_neighbors(x, y):
    return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]


bound = Bound(len(grid), len(grid[0]))
heap = [(0, (0, 0))]
min_cost = [
    [float('inf') for _ in row]
    for row in grid
]

while heap:
    cost, (x, y) = heappop(heap)
    if (x, y) not in bound or min_cost[x][y] <= cost:
        continue

    min_cost[x][y] = cost
    if (x, y) == (bound.x - 1, bound.y - 1):
        print(cost)
        break
    for i, j in get_neighbors(x, y):
        if (i, j) in bound:
            heappush(heap, (cost + grid[i][j], (i, j)))

2838
